## Missing Values

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [4]:
# 将文件路径保存到变量中，以便更方便地访问
melbourne_file_path = r'C:\Users\36085\Desktop\Python code\data\melb_data.csv'  # 输入数据地址 (在路径前面加r，即保持字符原始值的意思)

# 读取数据并将数据存储在名为"melbourne_data"的DataFrame中
data = pd.read_csv(melbourne_file_path) 
melb_predictors = data.drop(['Price'], axis=1)        # .drop()删除行列
X = melb_predictors.select_dtypes(exclude=['object']) # 选择除数据类型为object的列之外的所有列 (剔去文字)

y = data.Price
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

X   # 数据中存在NaN

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
0,2,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,-37.79960,144.99840,4019.0
1,2,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,-37.80790,144.99340,4019.0
2,3,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,-37.80930,144.99440,4019.0
3,3,2.5,3067.0,3.0,2.0,1.0,94.0,NaN,NaN,-37.79690,144.99690,4019.0
4,4,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,-37.80720,144.99410,4019.0
...,...,...,...,...,...,...,...,...,...,...,...,...
13575,4,16.7,3150.0,4.0,2.0,2.0,652.0,NaN,1981.0,-37.90562,145.16761,7392.0
13576,3,6.8,3016.0,3.0,2.0,2.0,333.0,133.0,1995.0,-37.85927,144.87904,6380.0
13577,3,6.8,3016.0,3.0,2.0,4.0,436.0,NaN,1997.0,-37.85274,144.88738,6380.0
13578,4,6.8,3016.0,4.0,1.0,5.0,866.0,157.0,1920.0,-37.85908,144.89299,6380.0


**Score from Approach 1 (Drop Columns with Missing Values)**

In [5]:
# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [6]:
#删除残缺列操作
cols_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()] # 存在nan就删除列

reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

183550.22137772635


**Score from Approach 2 (Imputation)**

In [7]:
from sklearn.impute import SimpleImputer

my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Imputation 移去了列名; put them back
imputed_X_train.columns = X_train.columns           # 用列名指定列名
imputed_X_valid.columns = X_valid.columns

print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

178166.46269899711


In [8]:
# Shape of training data (num_rows, num_columns)
print(X_train.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

X_train.isnull().sum() # 找缺失的列

(10864, 12)
Car               49
BuildingArea    5156
YearBuilt       4307
dtype: int64


Rooms               0
Distance            0
Postcode            0
Bedroom2            0
Bathroom            0
Car                49
Landsize            0
BuildingArea     5156
YearBuilt        4307
Lattitude           0
Longtitude          0
Propertycount       0
dtype: int64

## Categorical Variables

**One-Hot Encoding(独热码)** 创建新列，指示原始数据中每个可能值的存在(或不存在)

<img src="./photos/9.png" width="80%">